# Data analysis of the fantasy premier league

### Goal of Analysis: Use data science to get the best possible team for the next gameweek as well as suggest the best possible transfers according to the available resources.


In [2]:
import requests

FPL_URL = "https://fantasy.premierleague.com/api/bootstrap-static/"


def get_game_data():
    # get json object from fpl api
    data = requests.get(FPL_URL).json()

    return data

